In [ ]:
!pip install unetseg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Entrenamiento

In [ ]:
from unetseg.train import TrainConfig, train
from unetseg.evaluate import plot_data_generator
import os

In [ ]:
!ls /content/drive/MyDrive/Basurales/AMBA/labels/

In [ ]:
# Movemos a carpeta local para acelerar el entrenamiento
!cp -r $chips_folder /tmp/

In [ ]:
local_chips_folder = "/tmp/CHIPS/"

In [ ]:
# Cuantas imágenes para entrenar ("chips") tenemos?
!ls $local_chips_folder/images/ | wc -l

In [ ]:
# Aqui va la cantidad de imágenes / chips disponibles
n_img = 14209

# batch size: cantidad de datos que se procesan por vez, puede ser limitado por 
# la memoria de gpu disponible 
batch = 128

# steps per epoch: típicamente debe ser igual al numero de imágenes / el batch_size 
# (con un 10% arriba esta bien), si es mayor incrementara el número de imágenes 
# generadas con image augmentation
steps = round((n_img / batch ) * 1.1)

# tamaño de la imagen procesada por la UNet (debe ser multiplos de 16 , por ej 160, 320, etc) y lo mas parecida posible a la resolucion de los chips
unet_input_size = 160

# donde guardar el modelo (ruta y nombre de archivo) // guardamos en drive para preservarlo si se apaga la instancia de colab
model_file = '/microbasurales/models/2023_01_chipsize_160_48.h5'

config = TrainConfig(width=unet_input_size, 
                     height=unet_input_size,
                     n_channels=3,  #  número de canales de la imagen, rgb -> 3
                     n_classes=1, # número de clases a clasificar
                     apply_image_augmentation=True, #  si es True , amplia el dataset generando imagenes nuevas a partir de pequeñas variaciones de las ya existentes 
                     seed=42,
                     epochs=300, # Cantidad de veces que el dataset entero puede pasar por el proceso de entrenamiento
                     batch_size=batch, 
                     steps_per_epoch=steps, 
                     early_stopping_patience=60, # a medida que entrena se guarda los resultados del entrenamiento despues de cada epoch, si el error no varió luego de N iteraciones , se corta el proceso porque se entiende que el error ya disminuyó significativamente 
                     validation_split=0.2, # se divide la muestra en training y validation para calcular el error durante el proceso de entrenamiento
                     test_split=0.1,
                     images_path=local_chips_folder, #ruta a las imágenes
                     model_path=model_file, # ruta donde guardar el modelo entrenado
                     model_architecture='unet',
                     class_weights = [1],
                     evaluate=True) 

# con clases desbalanceadas se puede agregar el parámetro class_weights para darle mas peso a las clases minoritarias
# p.e.  class_weights = [0.8, 0.2]


In [ ]:
plot_data_generator(num_samples=5, fig_size=(10, 10), train_config=config, img_ch = 3)


In [ ]:
res_config = train(config)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,4))

plt.subplot(121)
plt.plot(res_config.history['loss'])
plt.plot(res_config.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(122)
plt.plot(res_config.history['mean_io_u_2'])
plt.plot(res_config.history['val_mean_io_u_2'])
plt.title('mean_iou')
plt.ylabel('val_mean_iou')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')


plt.show()